First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import cv2
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [2]:
model.load_state_dict(torch.load('road_following_model_white_ep12.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [4]:
torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

Load the optimized model by executing the cell below

In [5]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))#モデル読み込み

<All keys matched successfully>

Create the racecar class

In [6]:
from nvidia_racecar import NvidiaRacecar
#from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

Create the camera class.

In [7]:
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [8]:
from utils import preprocess
import numpy as np
import time

STEERING_GAIN = -1.5  #どの程度曲がるか
STEERING_BIAS = -0.0 #ステアリングの中心
THROTTLE_GAIN = -0.45 #スロットルの強さ
#THROTTLE_BIAS = -0.2

time_2=0
car.throttle = -0.45  #前進する強さ(初期値)
while True:
    for i in range(5):
        camera.read()
    image = camera.read()
    image = preprocess(image).half()#浮動小数点に変換
    label = model_trt(image).detach().cpu().numpy().flatten()
    time_2=time_2 + 1
    if time_2 == 100:
        print(lavel)
        time_2=0
    
    if label == 0:
      car.steering = 0.0
      car.throttle = -0.40
    elif label == 1:
      car.steering = 0.5
      car.throttle = -0.35
    elif label == 2:
      car.steering = 1.0
      car.throttle = -0.35
    elif label == 3:
      car.steering = 0.5
      car.throttle = 0.35
    elif label == 4:
      car.steering = 0.0
      car.throttle = 0.40
    elif label == 5:
      car.steering = -0.5
      car.throttle = 0.35
    elif label == 6:
      car.steering = -1.0
      car.throttle = -0.35
    elif label == 7:
      car.steering = -0.5
      car.throttle = -0.35
    #x = float(output[0])
    #car.steering = x
    #car.steering = (x) * STEERING_GAIN + STEERING_BIAS
    #if x >= 0.4 or x <= -0.4:
    #    car.throttle = THROTTLE_GAIN * 0.9
    #else:
    #    car.throttle = THROTTLE_GAIN

[-0.2108 -0.1674]
[-0.5957  -0.06003]
[-0.7896 -0.0268]
[ 1.061   -0.04947]
[ 0.09656 -0.0978 ]
[-0.02393  0.0788 ]
[-0.3257 -0.1656]


KeyboardInterrupt: 

In [10]:
#終了
car.throttle = 0.0
car.steering = 0.0